The aim of this notebook is to understand the various learnt latent factors.

In [1]:
import numpy as np
import pandas as pd
import sys
from common import compute_rmse_fraction


In [7]:
from mf_core import *

import numpy as np
import pandas as pd
import sys
from create_matrix import *
from sklearn.model_selection import LeaveOneOut

import pickle
from tensor_custom_core import *
from degree_days import dds

In [15]:
def learn_lf(region, year, a, b):
    APPLIANCES = ['fridge', 'hvac', 'wm', 'mw', 'oven', 'dw']


    def un_normalize(x, maximum, minimum):
        return (maximum-minimum)*x + minimum





    pred = {}
    lat = {}
    for appliance in APPLIANCES[:]:
        lat[appliance] = {}
        pred[appliance] = {}
        if appliance == "hvac":
            start, stop = 5, 11
        else:
            start, stop = 1, 13
        months = stop - start
        pred[appliance] = {}
        appliance_df = create_matrix_region_appliance_year(region, year, appliance, all_features=False)
        aggregate_cols = [x for x in appliance_df.columns if "aggregate" in x]
        appliance_cols = [x for x in appliance_df.columns if appliance in x]
        energy_cols = np.concatenate([aggregate_cols, appliance_cols])

        df = appliance_df.copy()
        dfc = df.copy()
        static_cols = ['area', 'total_occupants', 'num_rooms']
        static_df = df[static_cols]
        static_df = static_df.div(static_df.max())
        weather_values = np.array(dds[2014]['Austin'][start-1:stop-1]).reshape(-1,1)

        df = df[energy_cols]
        col_max = df.max().max()
        col_min = df.min().min()
        df = (1.0*(df-col_min))/(col_max-col_min)
        print df.describe()
        tensor = df.values.reshape((len(df), 2, months))
    
        M, N, O = tensor.shape
        print M, N, O
        mask = np.ones(M).astype('bool')

        case=2
        pred[appliance][case] = {}
        for a in range(1, 6):
            lat[appliance][a] = {}
            print "*"*20
            print a, case, appliance
            print "*"*20

            b = a
            pred[appliance][case][a] = {}
            for i, home in enumerate(df.index[:1]):


                tensor_copy = tensor.copy()
                tensor_copy[i, 1, :]=np.NaN
                H, A, T = learn_HAT(case, tensor_copy, a, a, num_iter=2000, lr=0.1, dis=False, cost_function='abs',
                                    H_known=static_df.values[:,:a])
                lat[appliance][a]['T'] = T
                lat[appliance][a]['A'] = A
                lat[appliance][a]['H'] = H
    return lat


In [16]:
%matplotlib inline

In [17]:
lat_sd = learn_lf('SanDiego', 2014, 3, 3)

       aggregate_1  aggregate_2  aggregate_3  aggregate_4  aggregate_5  \
count    20.000000    25.000000    27.000000    28.000000    35.000000   
mean      0.194262     0.166073     0.242901     0.242264     0.307896   
std       0.120995     0.108485     0.139808     0.142373     0.176196   
min       0.000779     0.002375     0.075035     0.065836     0.077360   
25%       0.121365     0.124581     0.139429     0.132520     0.173521   
50%       0.156221     0.143205     0.185854     0.211941     0.253810   
75%       0.317446     0.236359     0.321888     0.338436     0.407706   
max       0.371785     0.350720     0.707648     0.699491     0.774972   

       aggregate_6  aggregate_7  aggregate_8  aggregate_9  aggregate_10  \
count    35.000000    37.000000    36.000000    33.000000     30.000000   
mean      0.289319     0.400148     0.442438     0.488348      0.326873   
std       0.181512     0.231807     0.252178     0.269567      0.156596   
min       0.086280     0.022727  

********************
2 2 mw
********************
********************
3 2 mw
********************
********************
4 2 mw
********************
********************
5 2 mw
********************
       aggregate_1  aggregate_2  aggregate_3  aggregate_4  aggregate_5  \
count    20.000000    25.000000    27.000000    28.000000    35.000000   
mean      0.194401     0.166217     0.243032     0.242395     0.308016   
std       0.120974     0.108467     0.139783     0.142348     0.176165   
min       0.000952     0.002548     0.075195     0.065998     0.077520   
25%       0.121517     0.124733     0.139578     0.132670     0.173664   
50%       0.156368     0.143354     0.185995     0.212077     0.253939   
75%       0.317564     0.236491     0.322006     0.338551     0.407809   
max       0.371894     0.350833     0.707698     0.699543     0.775011   

       aggregate_6  aggregate_7  aggregate_8  aggregate_9  aggregate_10  \
count    35.000000    37.000000    36.000000    33.000000     

In [18]:
lat_austin = learn_lf('Austin', 2014, 3, 3)

       aggregate_1  aggregate_2  aggregate_3  aggregate_4  aggregate_5  \
count   375.000000   408.000000   427.000000   442.000000   452.000000   
mean      0.056781     0.048721     0.048082     0.051306     0.064090   
std       0.044105     0.038275     0.034509     0.039602     0.051570   
min       0.000160     0.000675     0.002178     0.000536     0.001207   
25%       0.032395     0.027567     0.028598     0.026551     0.032887   
50%       0.046410     0.039979     0.040509     0.043116     0.053317   
75%       0.068539     0.059921     0.061515     0.066765     0.082747   
max       0.521762     0.444762     0.429724     0.514999     0.655880   

       aggregate_6  aggregate_7  aggregate_8  aggregate_9  aggregate_10  \
count   465.000000   480.000000   490.000000   493.000000    491.000000   
mean      0.087652     0.098946     0.110153     0.086053      0.069450   
std       0.060303     0.067972     0.076722     0.059139      0.048184   
min       0.005484     0.000488  

********************
2 2 mw
********************
********************
3 2 mw
********************
********************
4 2 mw
********************
********************
5 2 mw
********************
       aggregate_1  aggregate_2  aggregate_3  aggregate_4  aggregate_5  \
count   375.000000   408.000000   427.000000   442.000000   452.000000   
mean      0.056781     0.048721     0.048082     0.051306     0.064090   
std       0.044105     0.038275     0.034509     0.039602     0.051570   
min       0.000160     0.000675     0.002178     0.000536     0.001207   
25%       0.032395     0.027567     0.028598     0.026551     0.032887   
50%       0.046410     0.039979     0.040509     0.043116     0.053317   
75%       0.068539     0.059921     0.061515     0.066765     0.082747   
max       0.521762     0.444762     0.429724     0.514999     0.655880   

       aggregate_6  aggregate_7  aggregate_8  aggregate_9  aggregate_10  \
count   465.000000   480.000000   490.000000   493.000000    4

In [19]:
import seaborn as sns

In [20]:
dds[2014]['Austin']

[0.012706480304955527,
 0.07369758576874205,
 0.09148665819567979,
 0.30495552731893266,
 0.5184243964421855,
 0.832274459974587,
 0.9085133418043202,
 1.0,
 0.747141041931385,
 0.4650571791613723,
 0.054637865311308764,
 0.03430749682337993]

In [21]:
import matplotlib.pyplot as plt

In [22]:
lat = {'Austin':lat_austin,'SanDiego':lat_sd}

In [23]:
def append_weather(df, appliance, region):
    if appliance=="hvac":
        start, stop = 5, 11
    else:
        start, stop = 1, 13
    degree_days = dds[2014][region][start-1:stop-1]
    df.loc['Degree Days \n(Normalised)'] = degree_days
    return df

def append_mean_energy(df, appliance):
    appliance_df = create_matrix_region_appliance_year(region, year, appliance,all=True)
    appliance_cols = [x for x in appliance_df.columns if appliance in x]
    appliance_df = appliance_df[appliance_cols]
    appliance_df.columns = range(len(appliance_cols))
    appliance_energy = appliance_df.mean().div(appliance_df.mean().max())
    df.loc['Mean Energy \n(Normalised)'] = appliance_energy
    return df
    

def plot_heatmap_correlation(appliance="fridge", num_factors=1, region="Austin"):    
    appliance="fridge"
    num_factors=1
    df = pd.DataFrame(lat[region][appliance][num_factors]['T']).T
    df.index = ['Latent_'+str(x) for x in df.index]
    df = append_weather(df, appliance, 'SanDiego')
    df = append_mean_energy(df, appliance)

    sns.heatmap(df, annot=True)
    plt.figure()
    sns.heatmap(df.T.corr(), annot=True)

In [24]:
plot_heatmap_correlation('hvac',1,'SanDiego')

NameError: global name 'region' is not defined

In [34]:
lat_sd['fridge'][2]['A'][:]

array([[[ 0.58268356,  0.24794032],
        [ 0.        ,  0.        ]],

       [[ 0.03299366,  0.03070862],
        [ 0.02261288,  0.00664193]]])

In [35]:
lat_austin['fridge'][2]['A'][:]

array([[[ 0.42216278,  0.30765528],
        [ 0.01273645,  0.00666931]],

       [[ 0.01352176,  0.01374141],
        [ 0.00646935,  0.0023954 ]]])

In [27]:
lat_sd['hvac'][2]['A'][:]

array([[[ 0.17401362,  0.44444258],
        [ 0.32969161,  0.2136885 ]],

       [[ 0.        ,  0.19034196],
        [ 0.06043612,  0.18822535]]])

In [28]:
lat_austin['hvac'][2]['A'][:]

array([[[ 0.25694309,  0.64890029],
        [ 0.00293389,  0.00953098]],

       [[ 0.04265277,  0.24999632],
        [ 0.00190629,  0.03492206]]])

In [31]:
lat_austin['hvac'][2]['A'][1:]

array([[[ 0.04265277,  0.24999632],
        [ 0.00190629,  0.03492206]]])